In [67]:
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset,DataLoader
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


iris_data = load_iris()

print(iris_data.data.shape)

(150, 4)


In [68]:
print(iris_data.target.shape)

(150,)


In [69]:
X = iris_data.data

y = iris_data.target

In [70]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2)


In [71]:
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [72]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

X_train_tensor = torch.tensor(X_train_scaled,dtype=torch.float32).to(device)
X_test_tensor = torch.tensor(X_test_scaled,dtype=torch.float32).to(device)

# 다중 분류의 loss 함수의 차이 때문에 y를 int 로 바꾼다
y_train_tensor = torch.tensor(y_train,dtype=torch.int).to(device)
y_train_tensor = y_train_tensor.long()
y_test_tensor = torch.tensor(y_test,dtype=torch.int).to(device)


## 배치사이즈 

In [73]:
train_dataset = TensorDataset(X_train_tensor,y_train_tensor)
train_loader = DataLoader(train_dataset,batch_size=128,shuffle=True)


In [ ]:

class Iris_Classifier(nn.Module):

    def __init__(self,input_dim,output_dim):
        super().__init__()
        self.linear1 = nn.Linear(input_dim,32)
        self.linear2 = nn.Linear(32,10)
        self.output = nn.Linear(10,output_dim)
        self.relu = nn.ReLU()
        self.log_softmax = nn.LogSoftmax(dim=1)

    def forward(self,x):
        x = self.relu(self.linear1(x))
        x = self.relu(self.linear2(x))
        x = self.output(x)
        return x


In [75]:

input_dim = X_train_tensor.shape[1]
output_dim = len(torch.unique(y_train_tensor))
model = Iris_Classifier(input_dim=input_dim,output_dim=output_dim).to(device)


In [76]:
criterion = nn.NLLLoss() # 다중 분류는 NLLoss 와 CrossEntropyLoss 를 주로 사용

optimizer = torch.optim.Adam(model.parameters(),lr = 0.01)

scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
            optimizer,mode='max',factor=0.5,patience=3,verbose= True   
) # mode : 높을수록 좋을때 , 반대는 min
  # factor :  러닝레이트를 몇 % 줄일거냐
  # patience : 몇번 참을 것인가.
  # verbose :  lr 이 변경 될 때 출력해줘

c:\Users\main\miniforge3\Lib\site-packages\torch\optim\lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


In [77]:
nb_epoch = 50
best_acc = 0
cnt = 0
patience = 5

for epoch in range(nb_epoch+1):
    # 학습 할거다 선언
    model.train()

    for batch_x,batch_y in train_loader:
        pred = model(batch_x)

        loss = criterion(pred,batch_y)

        optimizer.zero_grad()

        loss.backward()

        optimizer.step()

    model.eval()

    with torch.no_grad():
        test_pred = model(X_test_tensor)
        pred_trgt = torch.argmax(test_pred,axis=1)
        acc = (pred_trgt == y_test_tensor).float().mean().item()

        scheduler.step(acc)
        print(f'{epoch} / {nb_epoch} | Test acc : {acc * 100:.2f}%')

        # 조기종료

        if acc > best_acc :
            best_acc = acc
            cnt = 0
            torch.save(model.state_dict(),"best_iris_model.pt")
        else :
            cnt += 1
        if cnt >= patience:
            print('조기종료')    
            break;
print(f'최종 정확도 : {best_acc}')

0 / 50 | Test acc : 20.00%
1 / 50 | Test acc : 53.33%
2 / 50 | Test acc : 66.67%
3 / 50 | Test acc : 76.67%
4 / 50 | Test acc : 80.00%
5 / 50 | Test acc : 83.33%
6 / 50 | Test acc : 86.67%
7 / 50 | Test acc : 83.33%
8 / 50 | Test acc : 83.33%
9 / 50 | Test acc : 86.67%
10 / 50 | Test acc : 83.33%
11 / 50 | Test acc : 83.33%
조기종료
최종 정확도 : 0.8666667342185974


In [78]:
test_pred = model(X_test_tensor)

test_pred.shape # 데이터의 개수 X target count

test_pred

tensor([[-0.1801, -2.7741, -2.2790],
        [-3.4610, -1.3305, -0.3506],
        [-2.4552, -0.6600, -0.9231],
        [-2.0534, -0.7971, -0.8649],
        [-0.1369, -2.8310, -2.6738],
        [-2.8662, -1.0945, -0.4969],
        [-1.9861, -0.4462, -1.5019],
        [-3.6756, -1.2835, -0.3601],
        [-1.8132, -0.6395, -1.1734],
        [-0.1458, -2.7224, -2.6599],
        [-0.1772, -2.5440, -2.4792],
        [-0.1454, -3.0074, -2.4542],
        [-4.1154, -1.4632, -0.2848],
        [-2.5620, -0.6213, -0.9529],
        [-2.7949, -1.4129, -0.3632],
        [-3.0071, -1.2842, -0.3950],
        [-1.5394, -0.5310, -1.6221],
        [-0.1810, -2.8109, -2.2501],
        [-0.1988, -2.7442, -2.1540],
        [-2.2064, -0.8399, -0.7806],
        [-3.0485, -1.4517, -0.3307],
        [-1.7124, -0.7188, -1.1019],
        [-2.6272, -1.1306, -0.5027],
        [-2.6466, -0.8571, -0.6837],
        [-0.1624, -2.6199, -2.5630],
        [-0.1504, -2.7397, -2.5898],
        [-0.1788, -2.7995, -2.2740],
 

In [79]:
torch.argmax(test_pred,axis=1)

tensor([0, 2, 1, 1, 0, 2, 1, 2, 1, 0, 0, 0, 2, 1, 2, 2, 1, 0, 0, 2, 2, 1, 2, 2,
        0, 0, 0, 2, 1, 2], device='cuda:0')

In [80]:
(pred_trgt == y_test_tensor).float().mean().item()

0.8333333730697632